In [1]:
from courseraRec import fetchCourse

api_key = 'Esjk60d1dEckEaCwKRHfwwSm7aHEV04984NgANHe0cxeeoBt'
df = fetchCourse(api_key, 3000)


In [7]:
df.to_csv('coursera.csv')

In [1]:
import re
import nltk
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer


nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt_tab')
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\min\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\min\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\min\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\min\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from spellchecker import SpellChecker

DetectorFactory.seed = 0
english_words = set(word.lower() for word in words.words())
spell = SpellChecker(language='en')

def preprocess_text(text): # preprocessing for course name
    if not isinstance(text, str):
        return ''
    
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = nltk.word_tokenize(text)

    sentences = nltk.sent_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    english_sentences = []
    for sentence in sentences:
        try:
            if detect(sentence) == 'en':
                tokens = nltk.word_tokenize(sentence)
                tokens = [word for word in tokens if word not in stop_words]
                tokens = [lemmatizer.lemmatize(word) for word in tokens]
                tokens = [word for word in tokens if word in english_words]
                english_sentences.append(' '.join(tokens))
        except LangDetectException:
            continue

    return ' '.join(english_sentences)



In [8]:
import pandas as pd

coursera_df = pd.read_csv('coursera.csv')
coursera_df['preprocessed_desc'] = coursera_df['description'].apply(preprocess_text)
coursera_df.head()

,Unnamed: 0,id,name,slug,description,preprocessed_desc
0,0,DMkcgX7LEeyRTg6FtAvfBw,Intégrer des applications dans votre Dashboard...,integrer-applications-dashboard-hootsuite,"À la fin de ce projet, vous aurez toutes les c...",
1,1,ecoAPNowEe22LAo38arT7w,Overcoming Challenges in Self and Society,overcoming-challenges-in-self-and-society,This course is an personal development and aca...,course personal development academic voyage de...
2,2,YLO0oGSUEeyIUg4Qv2RsBQ,كيفية استعمال التطبيق هووتسويت,getting-started-with-hootsuite-ar,ينشئ العديد من الأشخاص حسابات وسائط اجتماعية م...,
3,3,69Bku0KoEeWZtA4u62x6lQ,Gamification,gamification,Gamification is the application of game elemen...,application game element digital game design t...
4,4,0HiU7Oe4EeWTAQ4yevf_oQ,Dealing With Missing Data,missing-data,This course will cover the steps used in weigh...,course cover step used weighting sample survey...


In [9]:
(coursera_df['preprocessed_desc'].values == '').sum() # 753 /3000 courses were empty

753

## Preprocessing skills, abilities and knowledge

In [2]:
import pandas as pd
agg_df = pd.read_csv('A:/UNI/website/Interest-seeker-Service/agg_df_scaled.csv')

# Define prefixes for different feature types
feature_prefixes = {
    'skills': 'skills_',
    'knowledge': 'knowledge_',
    'abilities': 'abilities_'
}

feature_names = {
    feature_type: [col.replace(prefix, '') for col in agg_df.columns if col.startswith(prefix)]
    for feature_type, prefix in feature_prefixes.items()
}

for feature_type, names in feature_names.items():
    print(f"{feature_type.capitalize()} Features:")
    print(names)
    print("\n")


Skills Features:
['Reading Comprehension', 'Active Listening', 'Writing', 'Speaking', 'Mathematics', 'Science', 'Critical Thinking', 'Active Learning', 'Learning Strategies', 'Monitoring', 'Social Perceptiveness', 'Coordination', 'Persuasion', 'Negotiation', 'Instructing', 'Service Orientation', 'Complex Problem Solving', 'Operations Analysis', 'Technology Design', 'Equipment Selection', 'Installation', 'Programming', 'Operations Monitoring', 'Operation and Control', 'Equipment Maintenance', 'Troubleshooting', 'Repairing', 'Quality Control Analysis', 'Judgment and Decision Making', 'Systems Analysis', 'Systems Evaluation', 'Time Management', 'Management of Financial Resources', 'Management of Material Resources', 'Management of Personnel Resources']


Knowledge Features:
['Administration and Management', 'Administrative', 'Economics and Accounting', 'Sales and Marketing', 'Customer and Personal Service', 'Personnel and Human Resources', 'Transportation', 'Production and Processing', 'F

In [21]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

stemmer = PorterStemmer()
stop_words = {'and', 'of'}

def preprocess_text_c(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    sentences = sent_tokenize(text)
    prep_sentences = []

    for sentence in sentences:
        words_sentence = word_tokenize(sentence)
        stemmed_words = [stemmer.stem(word) for word in words_sentence]
        filtered_words = [word for word in stemmed_words if word not in stop_words]
        if filtered_words:
            prep_sentences.append(' '.join(filtered_words))
    
    return ' '.join(prep_sentences)

In [29]:
processed_feature = {}
for feature_type, names in feature_names.items():
    processed_feature[feature_type] = [preprocess_text_c(name) for name in names]

for feature_type, names in processed_feature.items():
    print(f"Preprocessed {feature_type.capitalize()} Features:")
    print(names)
    print("\n")

Preprocessed Skills Features:
['read comprehens', 'activ listen', 'write', 'speak', 'mathemat', 'scienc', 'critic think', 'activ learn', 'learn strategi', 'monitor', 'social percept', 'coordin', 'persuas', 'negoti', 'instruct', 'servic orient', 'complex problem solv', 'oper analysi', 'technolog design', 'equip select', 'instal', 'program', 'oper monitor', 'oper control', 'equip mainten', 'troubleshoot', 'repair', 'qualiti control analysi', 'judgment decis make', 'system analysi', 'system evalu', 'time manag', 'manag financi resourc', 'manag materi resourc', 'manag personnel resourc']


Preprocessed Knowledge Features:
['administr manag', 'administr', 'econom account', 'sale market', 'custom person servic', 'personnel human resourc', 'transport', 'product process', 'food product', 'comput electron', 'engin technolog', 'design', 'build construct', 'mechan', 'mathemat', 'physic', 'chemistri', 'biolog', 'psycholog', 'sociolog anthropolog', 'geographi', 'medicin dentistri', 'therapi counsel

# Feature Extraction (BERT)

In [23]:
from sentence_transformers import SentenceTransformer
import pickle

model_name = 'all-MiniLM-L6-v2'
sentence_model = SentenceTransformer(model_name)
def compute_embeddings(data, model):
    embeddings = model.encode(data, show_progress_bar=True)
    return embeddings

a:\UNI\website\Interest-seeker-Service\.interest\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
# coursera embeddings
coursera_data = coursera_df['preprocessed_desc'].tolist()
coursera_embeddings = compute_embeddings(coursera_data, sentence_model)
coursera_df['embedding'] = list(coursera_embeddings)
with open('coursera_embeddings.pkl', 'wb') as f:
    pickle.dump(coursera_embeddings, f)

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

In [30]:
# skill, abilities, knowledge embedding
feature_embeddings = {}
for feature_type, texts in processed_feature.items():
    feature_embeddings[feature_type] = compute_embeddings(texts,sentence_model)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
with open('feature_embeddings.pkl', 'wb') as files:
    pickle.dump(feature_embeddings, files)

# Mapping Courses and Features

mapping each skill, knowledge area, and ability to the most relevant Coursera courses based on the similarity of their embeddings

In [16]:
# adding slug for easy approach
coursera_df = pd.read_csv('coursera.csv')
(coursera_df['slug'].values == '').sum()

0

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

with open('feature_embeddings.pkl', 'rb') as f:
    feature_embeddings = pickle.load(f)

with open('coursera_embeddings.pkl', 'rb') as f:
    coursera_embeddings = pickle.load(f)


def map_f_c(feature_embeddings, ogFname, course_names, course_slugs, course_embeddings, feature_type, top_n=5):
    mapping = {}
    feature_list = processed_feature[feature_type]
    embeddings = feature_embeddings[feature_type]

    similarity_m = cosine_similarity(embeddings, coursera_embeddings)
    topIunsort = np.argpartition(similarity_m, -top_n, axis=1)[:, -top_n:]
    sorted_top = np.argsort(similarity_m[np.arange(similarity_m.shape[0])[:,None], topIunsort], axis=1)[:,::-1] #*
    top_indices = topIunsort[np.arange(similarity_m.shape[0])[:,None],sorted_top]

    for original_name, indicies in zip(ogFname, top_indices):
        top_courses = [{'name': course_names[i], 'slug': course_slugs[i]} for i in indicies]
        mapping[original_name] = top_courses
    
    return mapping

In [33]:
course_embeddings = np.array(coursera_embeddings) # Shape: (n_courses, embedding_dim)
coursera_df = pd.read_csv('coursera.csv')

course_names = coursera_df['name'].tolist()
course_slug = coursera_df['slug'].tolist()

assert len(course_names) == course_embeddings.shape[0] == len(course_slug)

In [37]:
import pprint

top_n = 5
ogfn = feature_names['skills']
feature_to_courses = {}
feature_to_courses['skills'] = map_f_c(feature_embeddings, ogfn, course_names, course_slug, course_embeddings, 'skills', top_n=5)

pprint.pprint(feature_to_courses)

{'skills': {'Active Learning': [{'name': 'Discover Acting',
                                 'slug': 'discover-acting'},
                                {'name': 'Learn CSS Animations',
                                 'slug': 'learn-css-animations'},
                                {'name': 'Accelerate Your Learning with '
                                         'ChatGPT',
                                 'slug': 'learning-chatgpt'},
                                {'name': 'Connecting to Devices and Networks',
                                 'slug': 'codio-connecting-to-devices-and-networks'},
                                {'name': 'Gen AI for Software Development: '
                                         'Code Generation for Python',
                                 'slug': 'gen-ai-code-generation-for-python'}],
            'Active Listening': [{'name': 'Active Listening: Enhancing '
                                          'Communication Skills',
                            